In [ ]:
# Relative abundance barplots by taxonomic rank
# Author: Lucie Frejlichová
# Supplementary visualization for HTS fungal data

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Load and reshape data ===
df = pd.read_csv("data/TO_BE_FILLED_fungi_OTU_table_subset.csv", sep=';')

df_long = df.melt(
    id_vars=['kingdom', 'phylum', 'class', 'order', 'family', 'genus'],
    var_name='SampleCode',
    value_name='Abundance'
)

# === 2. Map sample codes to grouped locations ===
location_mapping = {
    'N1B': ('N1', 'Base'), 'N1S': ('N1', 'Stem'),
    'N2B': ('N2', 'Base'), 'N2S': ('N2', 'Stem'),
    'N3B': ('N3', 'Base'), 'N3S': ('N3', 'Stem'),
    'N4B': ('N4', 'Base'), 'N4S': ('N4', 'Stem'),
    'N5B': ('N5', 'Base'), 'N5S': ('N5', 'Stem'),
    'N6B': ('N6', 'Base'), 'N6S': ('N6', 'Stem'),
    'N7B': ('N7', 'Base'), 'N7S': ('N7', 'Stem')
}
df_long['Location'] = df_long['SampleCode'].map(location_mapping)
df_long = df_long[df_long['Location'].notna()]

# === 3. Plotting function ===
def plot_relative_abundance(df_long, tax_rank, title):
    counts = df_long.groupby(['Location', tax_rank])['Abundance'].sum().unstack(fill_value=0)
    rel_abundance = counts.div(counts.sum(axis=1), axis=0) * 100
    total_abundance = rel_abundance.sum(axis=0).sort_values(ascending=False)
    taxa = total_abundance.index.tolist()

    num_taxa = len(taxa)
    palette = sns.color_palette("tab20") + sns.color_palette("tab20b") + sns.color_palette("tab20c") + sns.color_palette("Set3")
    extended_palette = (palette * ((num_taxa // len(palette)) + 1))[:num_taxa]
    color_dict = {taxon: extended_palette[i] for i, taxon in enumerate(taxa)}

    ax = rel_abundance[taxa].plot(
        kind='bar',
        stacked=True,
        figsize=(14, 8),
        color=[color_dict[t] for t in taxa],
        edgecolor='black',
        width=0.85
    )

    plt.title(title)
    plt.ylabel('Relative abundance [%]')
    plt.xlabel('Location')
    plt.xticks(rotation=45)

    top_20 = total_abundance.iloc[:20]
    top_20_labels = list(top_20.index)
    top_20_colors = [color_dict[tax] for tax in top_20.index]
    handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in top_20_colors]
    ax.legend(handles, top_20_labels, title=tax_rank, bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.tight_layout()
    plt.show()

# === 4. Generate plots ===
plot_relative_abundance(df_long, 'genus', 'Relative abundance by Genus (HTS)')
plot_relative_abundance(df_long, 'family', 'Relative abundance by Family (HTS)')
plot_relative_abundance(df_long, 'order', 'Relative abundance by Order (HTS)')
plot_relative_abundance(df_long, 'class', 'Relative abundance by Class (HTS)')
plot_relative_abundance(df_long, 'phylum', 'Relative abundance by Phylum (HTS)')

In [ ]:
# Alpha diversity analysis with Shannon, Simpson, and Chao1 indices
# Author: Lucie Frejlichová
# Supplementary script for diversity comparison across sites

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import entropy, mannwhitneyu

# === Data Paths ===
data_path = "data/TO_BE_FILLED_fungi_OTU_table_subset.csv"
diversity_metrics_out = "data/TO_BE_FILLED_diversity_indices_subset.csv"
p_values_out = "data/TO_BE_FILLED_p_values_between_sites.csv"

# === Load Data ===
df = pd.read_csv(data_path, sep=';')

# === Location Mapping ===
location_mapping = {
    'N1B': ('N1', 'Base'), 'N1S': ('N1', 'Stem'),
    'N2B': ('N2', 'Base'), 'N2S': ('N2', 'Stem'),
    'N3B': ('N3', 'Base'), 'N3S': ('N3', 'Stem'),
    'N4B': ('N4', 'Base'), 'N4S': ('N4', 'Stem'),
    'N5B': ('N5', 'Base'), 'N5S': ('N5', 'Stem'),
    'N6B': ('N6', 'Base'), 'N6S': ('N6', 'Stem'),
    'N7B': ('N7', 'Base'), 'N7S': ('N7', 'Stem')
}

# === Data Transformation ===
df_long = df.melt(id_vars=['kingdom', 'phylum', 'class', 'order', 'family', 'genus'],
                  var_name='Location', value_name='Abundance')
df_long['Location'] = df_long['Location'].str.strip()
df_long[['Site', 'Sample_Type']] = df_long['Location'].apply(lambda x: pd.Series(location_mapping.get(x, (x, 'Unknown'))))
df_long['Abundance'] = pd.to_numeric(df_long['Abundance'], errors='coerce')
df_long['Rel_Abund'] = df_long.groupby('Location')['Abundance'].transform(lambda x: x / x.sum() * 100)

# === Pivot for Diversity Metrics ===
diversity_input = df_long.pivot_table(index='Location', columns='genus', values='Abundance', fill_value=0)

# === Compute Diversity Metrics ===
diversity_metrics = pd.DataFrame({
    'Shannon': diversity_input.apply(lambda x: entropy(x, base=np.e), axis=1),
    'Simpson': diversity_input.apply(lambda x: 1 - sum((x / x.sum()) ** 2), axis=1),
    'Chao1': diversity_input.apply(lambda x: x.count() + (x.eq(1).sum() ** 2) / (2 * x.eq(2).sum() + 1e-6), axis=1)
})
diversity_metrics[['Site', 'Sample_Type']] = diversity_metrics.index.to_series().apply(
    lambda x: pd.Series(location_mapping.get(x, (x, 'Unknown')))
)

# === Save Diversity Metrics ===
diversity_metrics.to_csv(diversity_metrics_out, index=False)
print(f"✅ Diversity indices saved to: {diversity_metrics_out}")

# === Pairwise Comparison (p-values) ===
p_values = {}
metrics = ['Shannon', 'Simpson', 'Chao1']
sites = diversity_metrics['Site'].unique()

for i, site1 in enumerate(sites):
    for site2 in sites[i+1:]:
        subset1 = diversity_metrics[diversity_metrics['Site'] == site1]
        subset2 = diversity_metrics[diversity_metrics['Site'] == site2]
        p_values[(site1, site2)] = [
            mannwhitneyu(subset1[metric], subset2[metric], alternative='two-sided')[1]
            if len(subset1) > 1 and len(subset2) > 1 else np.nan
            for metric in metrics
        ]

# Save p-values
df_p = pd.DataFrame.from_dict(p_values, orient='index', columns=[f"{m}_p" for m in metrics])
df_p.to_csv(p_values_out)
print(f"✅ P-values between sites saved to: {p_values_out}")

# === Plot Alpha Diversity ===
sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
sns.boxplot(x='Site', y='Shannon', data=diversity_metrics, palette='tab10', ax=axes[0])
axes[0].set_title('Shannon diversity by Site')
axes[0].tick_params(axis='x', rotation=45)

sns.boxplot(x='Site', y='Simpson', data=diversity_metrics, palette='tab20', ax=axes[1])
axes[1].set_title('Simpson diversity by Site')
axes[1].tick_params(axis='x', rotation=45)

sns.boxplot(x='Site', y='Chao1', data=diversity_metrics, palette='Set3', ax=axes[2])
axes[2].set_title('Chao1 diversity by Site')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
sns.boxplot(x='Sample_Type', y='Shannon', data=diversity_metrics, palette='Set2', ax=axes[0])
axes[0].set_title('Shannon diversity by Sample Type')

sns.boxplot(x='Sample_Type', y='Simpson', data=diversity_metrics, palette='muted', ax=axes[1])
axes[1].set_title('Simpson diversity by Sample Type')

sns.boxplot(x='Sample_Type', y='Chao1', data=diversity_metrics, palette='pastel', ax=axes[2])
axes[2].set_title('Chao1 diversity by Sample Type')

plt.tight_layout()
plt.show()


In [ ]:
# Beta Diversity Analysis using Bray-Curtis and Jaccard
# Author: Lucie Frejlichová
# Supplementary script for PCoA visualization, PERMANOVA, ANOSIM, and clustering

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN
from matplotlib.patches import Ellipse
from skbio.stats.distance import DistanceMatrix, permanova, anosim
from skbio.stats.ordination import pcoa

# === Helper function: Confidence ellipse ===
def confidence_ellipse(x, y, ax, n_std=1.96, facecolor='none', **kwargs):
    if len(x) < 2:
        return
    cov = np.cov(x, y)
    if np.linalg.matrix_rank(cov) < 2:
        return
    lambda_, v = np.linalg.eig(cov)
    if np.any(lambda_ < -1e-10) or np.any(np.isnan(lambda_)):
        print("Warning: ellipse not drawn due to bad covariance matrix")
        return
    lambda_ = np.sqrt(np.abs(lambda_))
    ell = Ellipse(xy=(np.mean(x), np.mean(y)),
                  width=lambda_[0] * n_std * 2,
                  height=lambda_[1] * n_std * 2,
                  angle=np.rad2deg(np.arccos(v[0, 0])),
                  facecolor=facecolor, **kwargs)
    ax.add_patch(ell)

# === Load data ===
file_path = 'data/TO_BE_FILLED_fungi_OTU_table_subset.csv'
df = pd.read_csv(file_path, sep=';')

# === Location mapping ===
location_mapping = {
    'N1B': ('N1', 'Base'), 'N1S': ('N1', 'Stem'),
    'N2B': ('N2', 'Base'), 'N2S': ('N2', 'Stem'),
    'N3B': ('N3', 'Base'), 'N3S': ('N3', 'Stem'),
    'N4B': ('N4', 'Base'), 'N4S': ('N4', 'Stem'),
    'N5B': ('N5', 'Base'), 'N5S': ('N5', 'Stem'),
    'N6B': ('N6', 'Base'), 'N6S': ('N6', 'Stem'),
    'N7B': ('N7', 'Base'), 'N7S': ('N7', 'Stem')
}

# === Data transformation ===
df_long = df.melt(id_vars=['kingdom', 'phylum', 'class', 'order', 'family', 'genus'],
                  var_name='Location', value_name='Abundance')
df_long['Location'] = df_long['Location'].str.strip()
df_long['Site'] = df_long['Location'].map(location_mapping)

# === Abundance matrix ===
abundance_matrix = df_long.pivot_table(index='Location', columns='genus', values='Abundance', fill_value=0)
abundance_matrix = abundance_matrix.div(abundance_matrix.sum(axis=1), axis=0)

# === Bray-Curtis Distance ===
bray_matrix = squareform(pdist(abundance_matrix, metric='braycurtis'))
grouping = abundance_matrix.index.to_series().map(location_mapping).tolist()
dm_bray = DistanceMatrix(bray_matrix, ids=abundance_matrix.index.tolist())
df_group = pd.DataFrame({'Group': grouping}, index=abundance_matrix.index)

# === PERMANOVA and ANOSIM (Bray-Curtis) ===
bray_permanova = permanova(dm_bray, df_group, column='Group', permutations=999)
bray_anosim = anosim(dm_bray, grouping, permutations=999)

# === PCoA Bray-Curtis ===
pcoa_bray = pcoa(dm_bray)
bc_coords = pcoa_bray.samples[['PC1', 'PC2']].values
expl_var_bray = pcoa_bray.proportion_explained.to_numpy() * 100

# === Clustering ===
db = DBSCAN(eps=0.1, min_samples=2).fit(bc_coords)
bc_labels = db.labels_.astype(str)

# === Bray-Curtis PCoA plot ===
samples_df = pd.DataFrame({
    'PCoA1': bc_coords[:, 0],
    'PCoA2': bc_coords[:, 1],
    'Cluster': bc_labels,
    'Group': grouping,
    'Sample': abundance_matrix.index
})

plt.figure(figsize=(8, 6))
ax = plt.gca()
sns.scatterplot(data=samples_df, x='PCoA1', y='PCoA2', hue='Group', style='Group', s=100, ax=ax)
for clust in samples_df['Cluster'].unique():
    if clust == '-1': continue
    subset = samples_df[samples_df['Cluster'] == clust]
    confidence_ellipse(subset['PCoA1'], subset['PCoA2'], ax, edgecolor='gray', linestyle='--', alpha=0.5)
for _, row in samples_df.iterrows():
    ax.text(row['PCoA1'] + 0.01, row['PCoA2'], row['Sample'], fontsize=8)
plt.title(f'PCoA – Bray-Curtis\nPC1: {expl_var_bray[0]:.1f}%, PC2: {expl_var_bray[1]:.1f}%\n'
          f'PERMANOVA F={bray_permanova["test statistic"]:.2f}, p={bray_permanova["p-value"]:.3f}')
plt.xlabel('PCoA1')
plt.ylabel('PCoA2')
plt.tight_layout()
plt.show()

# === Bray-Curtis Heatmap ===
distance_df = pd.DataFrame(bray_matrix, index=abundance_matrix.index, columns=abundance_matrix.index)
distance_df.index = abundance_matrix.index.to_series().map(location_mapping)
distance_df.columns = abundance_matrix.index.to_series().map(location_mapping)
bc_avg = distance_df.groupby(level=0).mean().groupby(level=0, axis=1).mean()

plt.figure(figsize=(8, 6))
sns.heatmap(bc_avg, cmap='viridis', annot=True, fmt=".2f")
plt.title('Bray-Curtis Dissimilarity Between Sites')
plt.tight_layout()
plt.show()

# === Jaccard Analysis ===
binary_matrix = (abundance_matrix > 0).astype(int)
jaccard_matrix = squareform(pdist(binary_matrix, metric='jaccard'))
jacc_dm = DistanceMatrix(jaccard_matrix, ids=abundance_matrix.index.tolist())

jacc_permanova = permanova(jacc_dm, df_group, column='Group', permutations=999)
jacc_anosim = anosim(jacc_dm, grouping, permutations=999)

pcoa_jacc = pcoa(jacc_dm)
jacc_coords = pcoa_jacc.samples[['PC1', 'PC2']].values
expl_var_jacc = pcoa_jacc.proportion_explained.to_numpy() * 100

db = DBSCAN(eps=0.1, min_samples=2).fit(jacc_coords)
jacc_labels = db.labels_.astype(str)

jacc_df = pd.DataFrame({
    'PCoA1': jacc_coords[:, 0],
    'PCoA2': jacc_coords[:, 1],
    'Cluster': jacc_labels,
    'Group': grouping,
    'Sample': abundance_matrix.index
})

plt.figure(figsize=(8, 6))
ax = plt.gca()
sns.scatterplot(data=jacc_df, x='PCoA1', y='PCoA2', hue='Group', style='Group', s=100, ax=ax)
for clust in jacc_df['Cluster'].unique():
    if clust == '-1': continue
    subset = jacc_df[jacc_df['Cluster'] == clust]
    confidence_ellipse(subset['PCoA1'], subset['PCoA2'], ax, edgecolor='gray', linestyle='--', alpha=0.5)
for _, row in jacc_df.iterrows():
    ax.text(row['PCoA1'] + 0.01, row['PCoA2'] + 0.01, row['Sample'], fontsize=8)
plt.title(f'PCoA – Jaccard (Presence/Absence)\nPC1: {expl_var_jacc[0]:.1f}%, PC2: {expl_var_jacc[1]:.1f}%\n'
          f'PERMANOVA F={jacc_permanova["test statistic"]:.2f}, p={jacc_permanova["p-value"]:.3f}')
plt.xlabel('PCoA1')
plt.ylabel('PCoA2')
plt.tight_layout()
plt.show()

# === Jaccard Heatmap ===
jacc_df_full = pd.DataFrame(jaccard_matrix, index=abundance_matrix.index, columns=abundance_matrix.index)
jacc_df_full.index = abundance_matrix.index.to_series().map(location_mapping)
jacc_df_full.columns = abundance_matrix.index.to_series().map(location_mapping)
jacc_avg = jacc_df_full.groupby(level=0).mean().groupby(level=0, axis=1).mean()

plt.figure(figsize=(8, 6))
sns.heatmap(jacc_avg, cmap='viridis', annot=True, fmt=".2f")
plt.title('Jaccard Dissimilarity Between Sites')
plt.tight_layout()
plt.show()